In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
!wget https://f000.backblazeb2.com/file/malaya-model/v38/generator/base.pb
!wget https://f000.backblazeb2.com/file/malaya-model/v38/generator/small.pb

--2020-11-16 11:22:24--  https://f000.backblazeb2.com/file/malaya-model/v38/generator/base.pb
Resolving f000.backblazeb2.com (f000.backblazeb2.com)... 104.153.233.177
Connecting to f000.backblazeb2.com (f000.backblazeb2.com)|104.153.233.177|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 1252668691 (1.2G) [application/octet-stream]
Saving to: ‘base.pb’

base.pb             100%[===================>]   1.17G  13.0MB/s    in 1m 46s  

2020-11-16 11:24:12 (11.3 MB/s) - ‘base.pb’ saved [1252668691/1252668691]

--2020-11-16 11:24:12--  https://f000.backblazeb2.com/file/malaya-model/v38/generator/small.pb
Resolving f000.backblazeb2.com (f000.backblazeb2.com)... 104.153.233.177
Connecting to f000.backblazeb2.com (f000.backblazeb2.com)|104.153.233.177|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 355570391 (339M) [application/octet-stream]
Saving to: ‘small.pb’

small.pb            100%[===================>] 339.10M  11.2MB/s    in 31s     

20

In [3]:
import tensorflow as tf
from tensorflow.tools.graph_transforms import TransformGraph
from glob import glob
tf.compat.v1.set_random_seed(0)

In [4]:
pbs = glob('*.pb')
pbs

['small.pb', 'base.pb']

In [5]:
import tensorflow_text
import tf_sentencepiece

In [6]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_constants(ignore_errors=true)',
             'fold_batch_norms',
             'fold_old_batch_norms',
#              'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

for pb in pbs:
    input_graph_def = tf.compat.v1.GraphDef()
    with tf.compat.v1.gfile.FastGFile(pb, 'rb') as f:
        input_graph_def.ParseFromString(f.read())
        
    print(pb)
    
    transformed_graph_def = TransformGraph(input_graph_def, 
                                           ['inputs'],
                                           ['SentenceTokenizer_1/SentenceTokenizer/SentencepieceDetokenizeOp'], transforms)
    
    with tf.compat.v1.gfile.GFile(f'{pb}.quantized', 'wb') as f:
        f.write(transformed_graph_def.SerializeToString())

Instructions for updating:
Use tf.compat.v1.gfile.GFile.
small.pb
base.pb


In [7]:
def load_graph(frozen_graph_filename, **kwargs):
    with tf.compat.v1.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())

    # https://github.com/onnx/tensorflow-onnx/issues/77#issuecomment-445066091
    # to fix import T5
    for node in graph_def.node:
        if node.op == 'RefSwitch':
            node.op = 'Switch'
            for index in xrange(len(node.input)):
                if 'moving_' in node.input[index]:
                    node.input[index] = node.input[index] + '/read'
        elif node.op == 'AssignSub':
            node.op = 'Sub'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
        elif node.op == 'AssignAdd':
            node.op = 'Add'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
        elif node.op == 'Assign':
            node.op = 'Identity'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
            if 'validate_shape' in node.attr:
                del node.attr['validate_shape']
            if len(node.input) == 2:
                node.input[0] = node.input[1]
                del node.input[1]

    with tf.compat.v1.Graph().as_default() as graph:
        tf.compat.v1.import_graph_def(graph_def)
    return graph

In [8]:
# g = load_graph('base.pb.quantized')
# x = g.get_tensor_by_name('import/inputs:0')
# logits = g.get_tensor_by_name('import/SentenceTokenizer_1/SentenceTokenizer/SentencepieceDetokenizeOp:0')

In [28]:
# x, x_len, logits

In [9]:
# test_sess = tf.compat.v1.InteractiveSession(graph = g)

In [10]:
# x

In [11]:
# %%time
# test_sess.run(logits, feed_dict = {x: ['ringkasan: KUALA LUMPUR: Presiden Perancis Emmanuel Macron tidak menampakkan beliau seorang sosok yang bertamadun, selar Tun Dr Mahathir Mohamad menerusi kemas kini terbaharu di blognya. Bekas Perdana Menteri itu mendakwa, pemerintah tertinggi Perancis itu bersikap primitif kerana menuduh orang Islam terlibat dalam pembunuhan guru yang menghina Islam, malah menegaskan tindakan membunuh bukan ajaran Islam. Jelas Dr Mahathir, sejarah membuktikan bahawa orang Perancis pernah membunuh jutaan manusia, yang ramai mangsanya terdiri dari orang Islam.']})

In [12]:
# %%time
# test_sess.run(logits, feed_dict = {x: [[1,2,3,3,4]], x_len: [[1,1,1,1,1]]})

In [13]:
quantized = glob('*.pb.quantized')
quantized

['small.pb.quantized', 'base.pb.quantized']

In [16]:
!rm *.pb*

In [ ]:
# converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
#     graph_def_file='test.pb',
#     input_arrays=['Placeholder', 'Placeholder_1'],
#     input_shapes={'Placeholder' : [None, 512], 'Placeholder_1': [None, 512]},
#     output_arrays=['logits'],
# )
# # converter.allow_custom_ops=True

In [ ]:
# converter.target_spec.supported_ops = [tf.compat.v1.lite.OpsSet.TFLITE_BUILTINS, tf.compat.v1.lite.OpsSet.SELECT_TF_OPS]
# converter.target_spec.supported_types = [tf.compat.v1.float16]
# converter.optimizations = [tf.compat.v1.lite.Optimize.DEFAULT]
# converter.experimental_new_converter = True
# tflite_model = converter.convert()

In [ ]:
# converter.target_spec.supported_ops = [tf.compat.v1.lite.OpsSet.TFLITE_BUILTINS, 
#                                        tf.compat.v1.lite.OpsSet.SELECT_TF_OPS]
# converter.target_spec.supported_types = [tf.compat.v1.float16]
# converter.optimizations = [tf.compat.v1.lite.Optimize.DEFAULT]
# tflite_model = converter.convert()

# with open('tiny-bert-sentiment-float16.tflite', 'wb') as f:
#     f.write(tflite_model)

In [ ]:
# converter.target_spec.supported_ops = [tf.compat.v1.lite.OpsSet.TFLITE_BUILTINS, 
#                                        tf.compat.v1.lite.OpsSet.SELECT_TF_OPS]
# converter.optimizations = [tf.compat.v1.lite.Optimize.OPTIMIZE_FOR_SIZE]
# tflite_model = converter.convert()

# with open('tiny-bert-sentiment-hybrid.tflite', 'wb') as f:
#     f.write(tflite_model)

In [ ]:
# interpreter = tf.compat.v1.lite.Interpreter(model_path='tiny-bert-sentiment-hybrid.tflite')
# interpreter.allocate_tensors()